# Проект по SQL

## Описание проекта

Компания купила крупный сервис для чтения книг по подписке. Необходимо сформулировать ценностное предложение для нового продукта.Предоставлена база данных, в которой содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг

Цель исследования: проанализировать базу данных для формирования ценностного предложения

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры



## Исследование таблиц базы данных

In [2]:
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


База данных содержит следующую информацию и таблицы:
**Таблица books**
содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

**Таблица authors**
содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

**Таблица publishers**
содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

**Таблица ratings**
содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

**Таблица reviews**
содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

## Запросы к базе данных

### Количество книг, которое вышло после 1 января 2000 года

In [7]:
query = '''SELECT COUNT(book_id)
FROM books 
WHERE publication_date>'2000-01-01' '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


### Количество обзоров и средняя оценка для каждой книги

In [8]:
query = '''SELECT title,
COUNT (DISTINCT review_id) AS reviews_number,
ROUND(AVG(rating),2) AS rating_average
FROM books AS b
LEFT JOIN ratings AS r ON b.book_id=r.book_id
LEFT JOIN reviews AS rs ON b.book_id=rs.book_id
GROUP BY b.book_id
ORDER BY rating_average DESC, reviews_number DESC
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,title,reviews_number,rating_average
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,The Big Bad Wolf (Alex Cross #9),2,5.00
4,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


###  Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [9]:
query = '''
WITH publisher_books AS (
SELECT publisher,
COUNT(book_id) AS books_count
FROM publishers AS p
JOIN books AS b ON p.publisher_id=b.publisher_id
WHERE num_pages>50
GROUP BY publisher
ORDER BY books_count DESC)

SELECT publisher
FROM publisher_books
WHERE books_count= (SELECT MAX(books_count)
FROM publisher_books) '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher
0,Penguin Books


### Автор с самой высокой средней оценкой книг (среди книг с 50 и более оценками)

In [10]:
query = """

SELECT author,
AVG(rating) AS average_rating
FROM authors AS a
JOIN books AS b ON a.author_id = b.author_id
JOIN ratings AS r ON b.book_id = r.book_id

WHERE b.book_id IN (SELECT book_id
FROM ratings 
GROUP BY book_id
HAVING COUNT(rating_id) >= 50
ORDER BY COUNT(rating_id) DESC)
GROUP BY author
ORDER BY average_rating DESC
LIMIT 1"""
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.287097


### Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [11]:
query = """
SELECT ROUND(AVG(review_cnt))
FROM (SELECT username, 
COUNT(review_id) AS review_cnt
FROM reviews 
WHERE username IN (SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(rating_id) > 48)
GROUP BY  username
      ) AS rereview_count
"""
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,round
0,24.0


## Выводы

- **Количество книг, которое вышло после 1 января 2000 года:** После 1 января 2000 года вышло 819 книг
- **Количество обзоров и средняя оценка для каждой книги:** В отобранных данных содержится 1000 различных книг, при этом книга с наибольшим количеством обзоров и средним рейтингом - A Dirty Job (Grim Reaper #1)	
- **Издательство, которое выпустило наибольшее число книг толще 50 страниц** - Penguin Books
- **Автор с самой высокой средней оценкой книг (среди книг с 50 и более оценками)** - J.K. Rowling/Mary GrandPré со средней оценкой 4.287097
- **Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок:** В среднем, пользователи, которые поставили более 48 оценок пишут по 24 обзора